<a href="https://colab.research.google.com/github/akalyanam3/LLMagicians/blob/main/BaselineEvaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade transformers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="meta-llama/Llama-3.1-8B")

In [ ]:
!pip install datasets

from datasets import load_dataset

ds = load_dataset("MU-NLPC/Calc-mawps", "default")


In [ ]:
test_split = ds["test"][:]
fields = list(test_split)


In [ ]:
import time
time_arr = []
answer_arr = []
int_answer_arr = []
for i in range(len(test_split['id'])):
  prompt = test_split['question'][i] + " The answer to this question is:"

  start_time = time.time()
  generated_text = pipe(prompt)[0]['generated_text']
  end_time = time.time()

  time_arr.append(end_time - start_time)

  value = generated_text[len(prompt):].split()
  try:
    answer = eval(value[0])
    true_val = test_split['result_float'][i]
    if answer >= true_val - 0.1 and answer <= true_val + 0.1:
      answer_arr.append(1)
    else:
      answer_arr.append(0)
    if int(answer) == int(true_val):
      int_answer_arr.append(1)
    else:
      int_answer_arr.append(0)
  except:
    answer_arr.append(0)
    int_answer_arr.append(0)






In [38]:
print(time_arr)
print(answer_arr)
print(int_answer_arr)

print(len(time_arr))
print(len(answer_arr))
print(len(int_answer_arr))



[0.8053531646728516, 0.787766695022583, 0.7850368022918701, 0.7850189208984375, 0.7861218452453613, 0.749889612197876, 0.779202938079834, 0.7926905155181885, 0.7904167175292969, 0.7565865516662598, 0.5234313011169434, 0.785344123840332, 0.760573148727417, 0.7871263027191162, 0.7868869304656982, 0.7583684921264648, 0.7852232456207275, 0.7837400436401367, 0.7866158485412598, 0.785942792892456, 0.7807762622833252, 0.7842841148376465, 0.7827639579772949, 0.7784590721130371, 0.7745361328125, 0.7508759498596191, 0.7498219013214111, 0.7737770080566406, 0.7465841770172119, 0.7769675254821777, 0.592449426651001, 0.7803909778594971, 0.7684512138366699, 0.7797853946685791, 0.7454075813293457, 0.7077398300170898, 0.7860791683197021, 0.7934913635253906, 0.7566871643066406, 0.7846024036407471, 0.7341020107269287, 0.7474839687347412, 0.18940091133117676, 0.7738285064697266, 0.7475855350494385, 0.7799997329711914, 0.7808635234832764, 0.7477960586547852, 0.7790215015411377, 0.7478318214416504, 0.771937

In [47]:
answer_sum = 0
for answer in answer_arr:
  answer_sum += answer
print(answer_sum)

int_answer_sum = 0
for answer in int_answer_arr:
  int_answer_sum += answer
print(int_answer_sum)

time_sum = 0
for time in time_arr:
  time_sum += time
avg_time_ms = time_sum/520 * 1000


299
351


In [46]:
print(str(answer_sum*100/520) + "% accuracy rate when we consider integer responses for fractional/decimal answers as invalid.")
print(str(int_answer_sum*100/520) + "% accuracy rate when we consider integer responses for fractional/decimal answers as valid.")
print(str(avg_time_ms) + " ms average reasoning time, including pipeline connection time.")

57.5% accuracy rate when we consider integer responses for fractional/decimal answers as invalid.
67.5% accuracy rate when we consider integer responses for fractional/decimal answers as valid.
729.6377700108749 ms average reasoning time, including pipeline connection time.
